In [131]:
import yfinance as yf

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.base import clone

In [132]:
df_apple_to_csv = yf.download("AAPL", start="2020-01-01", end="2025-08-13")
df_microsoft_to_csv = yf.download("MSFT", start="2020-01-01", end="2025-08-13")
df_amazon_to_csv = yf.download("AMZN", start="2020-01-01", end="2025-08-13")


apple_path = 'datasets/apple.csv'
microsoft_path = 'datasets/microsoft.csv'
amazon_path = 'datasets/amazon.csv'

df_apple_to_csv.to_csv(apple_path)
df_microsoft_to_csv.to_csv(microsoft_path)
df_amazon_to_csv.to_csv(amazon_path)

df_apple = pd.read_csv(apple_path)
df_microsoft = pd.read_csv(microsoft_path)
df_amazon = pd.read_csv(amazon_path)

C:\Users\acer\AppData\Local\Temp\ipykernel_17368\1102566560.py:1: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed
C:\Users\acer\AppData\Local\Temp\ipykernel_17368\1102566560.py:2: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed
C:\Users\acer\AppData\Local\Temp\ipykernel_17368\1102566560.py:3: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed


In [133]:
df_apple.head()

,Price,Close,High,Low,Open,Volume
0,Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
1,Date,NaN,NaN,NaN,NaN,NaN
2,2020-01-02,72.53851318359375,72.59889150198023,71.29230380764884,71.54588979671196,135480400
3,2020-01-03,71.83329010009766,72.59405542623057,71.60868452797844,71.76566667933918,146322800
4,2020-01-06,72.4056625366211,72.44430553740506,70.7029972337209,70.95417305361724,118387200


In [134]:
df_microsoft.head()

,Price,Close,High,Low,Open,Volume
0,Ticker,MSFT,MSFT,MSFT,MSFT,MSFT
1,Date,NaN,NaN,NaN,NaN,NaN
2,2020-01-02,153.04229736328125,153.14710838701015,150.86034091881393,151.2891080017366,22622100
3,2020-01-03,151.13665771484375,152.4039130274651,150.60307959140601,150.8508226765117,21116200
4,2020-01-06,151.52728271484375,151.59398723405667,149.12616718464912,149.66928270769137,20813700


In [135]:
df_amazon.head()

,Price,Close,High,Low,Open,Volume
0,Ticker,AMZN,AMZN,AMZN,AMZN,AMZN
1,Date,NaN,NaN,NaN,NaN,NaN
2,2020-01-02,94.90049743652344,94.90049743652344,93.2074966430664,93.75,80580000
3,2020-01-03,93.74849700927734,94.30999755859375,93.2249984741211,93.2249984741211,75288000
4,2020-01-06,95.14399719238281,95.18450164794922,93.0,93.0,81236000


In [136]:
df_apple['Date'] = df_apple['Price']
df_apple = df_apple.drop(index=[0,1], columns='Price')

df_microsoft['Date'] = df_microsoft['Price']
df_microsoft = df_microsoft.drop(index=[0,1], columns='Price')

df_amazon['Date'] = df_amazon['Price']
df_amazon = df_amazon.drop(index=[0,1], columns='Price')

In [137]:
df_apple.head()

,Close,High,Low,Open,Volume,Date
2,72.53851318359375,72.59889150198023,71.29230380764884,71.54588979671196,135480400,2020-01-02
3,71.83329010009766,72.59405542623057,71.60868452797844,71.76566667933918,146322800,2020-01-03
4,72.4056625366211,72.44430553740506,70.7029972337209,70.95417305361724,118387200,2020-01-06
5,72.06513977050781,72.67133299565539,71.84536183598526,72.41532954893665,108872000,2020-01-07
6,73.22440338134766,73.5262949460089,71.76807876823776,71.76807876823776,132079200,2020-01-08


In [138]:
df_microsoft.head()

,Close,High,Low,Open,Volume,Date
2,153.04229736328125,153.14710838701015,150.86034091881393,151.2891080017366,22622100,2020-01-02
3,151.13665771484375,152.4039130274651,150.60307959140601,150.8508226765117,21116200,2020-01-03
4,151.52728271484375,151.59398723405667,149.12616718464912,149.66928270769137,20813700,2020-01-06
5,150.14573669433594,152.1371317710887,149.89800813543405,151.8036526962878,21634100,2020-01-07
6,152.5372772216797,153.21378726865242,150.49823863325935,151.43200016356872,27746500,2020-01-08


In [139]:
df_amazon.head()

,Close,High,Low,Open,Volume,Date
2,94.90049743652344,94.90049743652344,93.2074966430664,93.75,80580000,2020-01-02
3,93.74849700927734,94.30999755859375,93.2249984741211,93.2249984741211,75288000,2020-01-03
4,95.14399719238281,95.18450164794922,93.0,93.0,81236000,2020-01-06
5,95.34300231933594,95.69450378417969,94.60199737548828,95.2249984741211,80898000,2020-01-07
6,94.59850311279297,95.55000305175781,94.3219985961914,94.9020004272461,70160000,2020-01-08


In [140]:
df_apple.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1410 entries, 2 to 1411
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Close   1410 non-null   object
 1   High    1410 non-null   object
 2   Low     1410 non-null   object
 3   Open    1410 non-null   object
 4   Volume  1410 non-null   object
 5   Date    1410 non-null   object
dtypes: object(6)
memory usage: 66.2+ KB


In [141]:
df_apple.describe()

,Close,High,Low,Open,Volume,Date
count,1410,1410,1410,1410,1410,1410
unique,1394,1410,1410,1410,1408,1410
top,139.26516723632812,230.8000030517578,227.07000732421875,228.00999450683594,90956700,2025-08-12
freq,2,1,1,1,2,1


In [142]:
df_apple.isna().sum()

Close     0
High      0
Low       0
Open      0
Volume    0
Date      0
dtype: int64

In [143]:
#INITIAL
dfs = [df_apple, df_amazon, df_microsoft]
window = 14
short_period = 12
long_period = 26
signal_line_period = 9
years = list(range(2020,2026))
stocks = ['AAPL', 'AMZN', 'MSFT']
X_train, X_test, y_train, y_test = None, None, None, None

In [144]:
#CONVERTING TO CORRECT FEATURE TYPES
def convert_type(df):
    df['Close'] = df['Close'].astype(float)
    df['High'] = df['High'].astype(float)
    df['Low'] = df['Low'].astype(float)
    df['Open'] = df['Open'].astype(float)
    df['Volume'] = df['Volume'].astype(float)
    df['Date'] = pd.to_datetime(df['Date'])

In [145]:
#FEATURE ENGINEERING
def engineer_features(df):
    df['Price Change'] = df['Close'].diff().fillna(0)

    df['Gain'] = df.loc[df['Price Change'] > 0, 'Price Change']
    df['Loss'] = -df.loc[df['Price Change'] < 0, 'Price Change']

    df['Gain'] = df['Gain'].fillna(0)
    df['Loss'] = df['Loss'].fillna(0)

    df['Average Gain'] = df['Gain'].rolling(window=window).mean().fillna(0)
    df['Average Loss'] = df['Loss'].rolling(window=window).mean().fillna(0)

    df['RS'] = df['Average Gain'] / df['Average Loss']

    df['RSI'] = 100 - (100 / (1 + df['RS']))

    df['Short EMA'] = df['Close'].ewm(span=short_period,adjust=False).mean()
    df['Long EMA'] = df['Close'].ewm(span=long_period, adjust=False).mean()

    df['MACD'] = df['Short EMA'] - df['Long EMA']

    df['Signal Line'] = df['MACD'].ewm(span=signal_line_period, adjust=False).mean()
    
    df['MACD Histogram'] = df['MACD'] - df['Signal Line']
    
    df['Year'] = df['Date'].dt.year

In [146]:
#DROPPING NULL AND UNIMPORTANT COLUMNS/ROWS
def drop_features_and_na(df):
    df = df.drop(columns=['Gain', 'Loss'])
    df = df.dropna()
    return df

In [147]:
#PRINTING SEABORN PLOTS
def print_plots(df, stock):
    for year in years:
        sns.relplot(data=df.loc[df['Year'] == year], x='Date', y='RSI', kind='line')
        plt.title(f'RSI trend for year {year} Stock {stock}')
        plt.figure(figsize=(20,10))
        plt.show()
        
        sns.relplot(data=df.loc[df['Year'] == year], x='Date', y='MACD', kind='line')
        plt.title(f'MACD trend for year {year} Stock {stock}')
        plt.figure(figsize=(20,10))
        plt.show()

In [148]:
#MODEL TRAINING AND EVALUATION
def split_data(df):
    X = df.drop(columns=['Date', 'Close'])
    y = df['Close']

    return train_test_split(X, y, test_size=0.2, random_state=42)

In [149]:
#MAIN OUTPUT
for i,df in enumerate(dfs):
    convert_type(df)
    engineer_features(df)
    df = drop_features_and_na(df)
    
    print(f"{stocks[i]}:")
    print(df.head())
    print()
    
    # print_plots(df, stocks[i]) TODO: must implement merged stocks for good plotting visuals
    
    X_train, X_test, y_train, y_test = split_data(df)

AAPL:
        Close       High        Low       Open       Volume       Date  \
15  76.728767  77.281827  76.634574  76.941293  101832400.0 2020-01-22   
16  77.098289  77.177986  76.233668  76.781908  104472000.0 2020-01-23   
17  76.876099  78.088493  76.685301  77.344635  146537600.0 2020-01-24   
18  74.615532  75.296594  73.632571  74.883608  161940000.0 2020-01-27   
19  76.726349  76.897821  75.398026  75.497047  162234000.0 2020-01-28   

    Price Change  Average Gain  Average Loss        RS        RSI  Short EMA  \
15      0.272903      0.508214      0.208910  2.432695  70.868369  75.349468   
16      0.369522      0.534608      0.208910  2.559039  71.902524  75.618517   
17     -0.222191      0.534608      0.174407  3.065283  75.401465  75.811991   
18     -2.260567      0.493724      0.335876  1.469958  59.513481  75.627921   
19      2.110817      0.644497      0.311553  2.068656  67.412447  75.796910   

     Long EMA      MACD  Signal Line  MACD Histogram  Year  
15  74.

In [150]:
#TODO: BELOW THIS CELL SHOULD FIX MODEL SELECTION, PIPELINE, PREDICTING FOR EACH STOCKS

In [151]:
# model = Pipeline([
#     ('scaler', StandardScaler()),
#     ('lr', LinearRegression())
# ])

In [152]:
# kfold = KFold(n_splits=6, random_state=30, shuffle=True)

# params = {'lr__n_jobs': [10,20,50]}
# cv = GridSearchCV(estimator=model,param_grid=params,cv=kfold)

# cv.fit(X_train, y_train)

# print("Best parameter:", cv.best_params_)
# print("Best score:", cv.best_score_)


In [153]:
# y_pred = cv.predict(X_test)

# print("Predicted values respectively:",y_pred[0:10])

# print("Correct values:")
# print(y_test[0:10])